In [42]:
# !pip install numpy==1.19.5
import tensorflow as tf
import numpy as np
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [32]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

TypeError: 'type' object is not subscriptable

In [9]:
BATCH_SIZE = 32
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=50

In [10]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "Wheat Disease",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 2942 files belonging to 3 classes.


In [11]:
class_names = dataset.class_names
class_names

['Wheat___Brown_Rust', 'Wheat___Healthy', 'Wheat___Yellow_Rust']

In [12]:
len(dataset)

92

In [13]:
for image_batch, labels_batch in dataset.take(1):
    print(image_batch.shape)
    print(labels_batch.numpy())

(32, 256, 256, 3)
[1 0 1 0 2 1 2 0 2 2 2 1 0 2 1 0 1 2 2 1 0 2 1 1 0 2 2 1 1 0 1 1]


In [1]:
plt.figure(figsize=(10, 10))
for image_batch, labels_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[labels_batch[i]])
        plt.axis("off")

NameError: name 'plt' is not defined

In [14]:
train_size = 0.8
len(dataset)*train_size

73.60000000000001

In [15]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

In [16]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [17]:
len(train_ds)

73

In [18]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [19]:
resize_and_rescale = tf.keras.Sequential([
    tf.keras.layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    tf.keras.layers.Rescaling(1./255),
])


In [20]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
])

In [21]:
train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x, training=True), y)
).prefetch(buffer_size=tf.data.AUTOTUNE)

In [22]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 conv2d (Conv2D)             (32, 254, 254, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (32, 127, 127, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (32, 125, 125, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (32, 62, 62, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (32, 60, 60, 64)         

In [24]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [25]:
history = model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=val_ds,
    verbose=1,
    epochs=50,
)

Epoch 1/50
73/73 [==============================] - 44s 352ms/step - loss: 0.8036 - accuracy: 0.5724 - val_loss: 0.5703 - val_accuracy: 0.6736
Epoch 2/50
73/73 [==============================] - 17s 226ms/step - loss: 0.5861 - accuracy: 0.6932 - val_loss: 0.5617 - val_accuracy: 0.6840
Epoch 3/50
73/73 [==============================] - 17s 227ms/step - loss: 0.5916 - accuracy: 0.6855 - val_loss: 0.5492 - val_accuracy: 0.7500
Epoch 4/50
73/73 [==============================] - 17s 227ms/step - loss: 0.4596 - accuracy: 0.7926 - val_loss: 0.3300 - val_accuracy: 0.8507
Epoch 5/50
73/73 [==============================] - 17s 227ms/step - loss: 0.2997 - accuracy: 0.8732 - val_loss: 0.1882 - val_accuracy: 0.9340
Epoch 6/50
73/73 [==============================] - 17s 227ms/step - loss: 0.2632 - accuracy: 0.9049 - val_loss: 0.2575 - val_accuracy: 0.9340
Epoch 7/50
73/73 [==============================] - 17s 227ms/step - loss: 0.2152 - accuracy: 0.9340 - val_loss: 0.2454 - val_accuracy: 0.9167

In [33]:
scores = model.evaluate(test_ds)

10/10 [==============================] - 4s 43ms/step - loss: 0.0391 - accuracy: 0.9844


In [34]:
scores

[0.03907203674316406, 0.984375]

In [35]:
for images_batch, labels_batch in test_ds.take(1):
    
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()

In [36]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [38]:
# plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        # ax = plt.subplot(3, 3, i + 1)
        # plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        # plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        # plt.axis("off")
        print(f"actual_class {actual_class}, predicted_class {predicted_class}, confidence {confidence}")

1/1 [==============================] - 1s 674ms/step
actual_class Wheat___Brown_Rust, predicted_class Wheat___Brown_Rust, confidence 100.0
1/1 [==============================] - 0s 26ms/step
actual_class Wheat___Healthy, predicted_class Wheat___Healthy, confidence 100.0
1/1 [==============================] - 0s 26ms/step
actual_class Wheat___Yellow_Rust, predicted_class Wheat___Yellow_Rust, confidence 99.1
1/1 [==============================] - 0s 27ms/step
actual_class Wheat___Healthy, predicted_class Wheat___Healthy, confidence 100.0
1/1 [==============================] - 0s 34ms/step
actual_class Wheat___Healthy, predicted_class Wheat___Healthy, confidence 100.0
1/1 [==============================] - 0s 27ms/step
actual_class Wheat___Yellow_Rust, predicted_class Wheat___Yellow_Rust, confidence 99.94
1/1 [==============================] - 0s 25ms/step
actual_class Wheat___Yellow_Rust, predicted_class Wheat___Yellow_Rust, confidence 99.99
1/1 [==============================] - 0s 25ms

In [40]:
import os
model_version=max([int(i) for i in os.listdir("../models") + [0]])+1
model.save(f"../models/{model_version}")

INFO:tensorflow:Assets written to: ../models/1\assets


INFO:tensorflow:Assets written to: ../models/1\assets


In [1]:
model.save("./wheat.h5")

NameError: name 'model' is not defined